In [17]:
import pygame
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from PIL import Image
import random
import matplotlib.pyplot as plt
from matplotlib import animation
from collections import deque

In [18]:

print(torch.cuda.is_available())  # 應該為 True
print(torch.version.cuda)         # 應該列出 CUDA 版本
print(torch.backends.cudnn.version())  # cuDNN 版本

True
12.1
90100


In [19]:
script_dir = os.path.join(os.getcwd(), 'space_ship_game_RL')
if script_dir not in sys.path:
    sys.path.append(script_dir)

from setting import *
from game import Game


In [ ]:
class SpaceShipEnv():
    def __init__(self):
        pygame.init()
        pygame.font.init()

        # 延後畫面初始化，等 render() 時才設置
        self.screen = None
        self.clock = pygame.time.Clock()
        self.fps = FPS

        self.game = Game()

        self.action_space = [0, 1, 2, 3]
        self.observation = self.game.state
        self.prev_score = 0
        self.prev_health = self.game.player.sprite.health
        self.score_milestones = set()
        self.prev_combo = 0

        self.a_second_reward = 0.0
        self.a_second_score = 0

    def step(self, action):
        self.game.update(action)

        if self.screen is None:
            self.game.draw()
        else:
            self.game.draw(self.screen)
            self.clock.tick(self.fps)

        # define the state by your game logic
        state = self.game.state

        # define the reward by your game logic
        reward = 0.0
        # [98 84 40 43 26] 石頭半徑
        # [41.65 36.0 17.0 18.3 10.5] .radius = int(self.rect.width * 0.85 / 2)
        # 得分=半徑*2 傷害=半徑*1

        # 擊中石頭
        if self.game.is_hit_rock:
            hit_inc = self.game.score - self.prev_score
            reward +=  1 * hit_inc * (0.97 ** (self.game.frame_count // 3600)) 
            # print("Hit Rock Reward:", hit_inc, "Reward:", reward)

        # 連續擊中 combo bonus
        if self.game.combo >= 3 and self.game.combo > self.prev_combo:
            bonus = min(0.3 * (self.game.combo ** 1.3), 8)
            reward += bonus
            # print("Combo Bonus:", self.game.combo, "Reward:", reward)

        # 撿到護盾 / 武器
        if self.game.is_power_shield:
            reward += 6  # 護盾更高分
            # print("Shield Reward:", reward)
        if self.game.is_power_gun:
            reward += 5
            # print("Power-up Reward:", reward)

        # 撞到石頭（依損失血量懲罰）
        if self.game.is_collided:
            damage = self.game.is_collided_radius
            reward -= 0.4 * damage
            # print("Collision Damage:", damage, "Reward:", reward)
      
        # reward function 裡
        milestone = (self.game.score // 500) * 500
        if milestone >= 100 and milestone not in self.score_milestones:
            reward += max(1, 4.5 * np.exp(-self.game.score / 2000))
            self.score_milestones.add(milestone)
            # print("Score Milestone Reached:", milestone, "Reward:", reward)

        if self.game.frame_count % 20 == 0:
            # 懲罰（長時間未擊中）
            miss_penalty = 3 / (1 + np.exp(-0.02 * max(0, self.game.no_hit_steps - 50)))
            reward -= min(4.0, miss_penalty)
            # 生存微懲罰（與遊戲總時間成長）
            survival_penalty = 0.05
            reward -= survival_penalty

        # # 死亡懲罰
        # if not self.game.running:
        #     reward -= 100
        
        # 每秒的reward score 顯示
        self.a_second_reward += reward
        self.a_second_score += self.game.score - self.prev_score
        if not self.game.running:
            reward -= 100
            print(f"({self.game.frame_count / 60}) Reward : {self.a_second_reward:.2f} Score: {self.a_second_score}")
            self.a_second_reward = 0.0
            self.a_second_score = 0
        if self.game.frame_count % 60 == 0:
            print(f"({self.game.frame_count / 60}) Reward : {self.a_second_reward:.2f} Score: {self.a_second_score}")
            self.a_second_reward = 0.0
            self.a_second_score = 0

        reward = np.tanh(reward / 100.0) * 100
        self.prev_combo = self.game.combo
        self.prev_score = self.game.score
        self.prev_health = self.game.player.sprite.health
        # ------------------------------------
    
        done = not self.game.running or self.game.score >= 10000

        info = {
            "score": self.game.score,
            "health": self.game.player.sprite.health
        }
        
        return state, reward, done, info

    def reset(self):
        self.game = Game()
        self.prev_score = 0
        return self.game.state

    def render(self):
        if self.screen is None:
            self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
            pygame.display.set_caption("SpaceShip RL Environment")

    def close(self):
        pygame.quit()


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [22]:
class DDQN(nn.Module):
    def __init__(self, in_channels, num_actions):
        super(DDQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 512),
            nn.ReLU(),
            nn.Linear(512, num_actions)
        )

    def forward(self, x):
        x = self.conv(x / 255.0)
        return self.fc(x)


In [23]:
# Preprocess frames (grayscale and resize to 84x84)
# 預處理影格：轉為灰階並縮放為 84x84
import cv2
def preprocess_state(state, stacked_state=None, is_new=False):
    gray = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
    gray = torch.tensor(gray, dtype=torch.uint8).unsqueeze(0)        # (1,84,84)
    if is_new or stacked_state is None:
        stacked_state = gray.repeat(4, 1, 1)                         # (4,84,84)
    else:
        stacked_state = torch.cat((gray, stacked_state[:-1]), dim=0)
    return stacked_state.float().unsqueeze(0), stacked_state         # (1,4,84,84)


def preprocess_frame(frame):
    # frame 是 numpy array (H, W, 3)，先轉為 PIL Image
    # Input is a color image (RGB), convert to PIL format for easier processing.
    # 輸入是彩色圖像（RGB），轉成 PIL Image 以方便處理。
    image = Image.fromarray(frame)

    # 轉灰階
    # Convert the image to grayscale to reduce input complexity.
    # 將影像轉為灰階，降低輸入維度與計算量。
    image = image.convert('L')

    # resize 成 84x84
    # Resize the image to a standard 84x84 shape, as per DQN convention.
    # 依照 DQN 的慣例將影像統一縮放至 84x84。
    image = image.resize((84, 84), Image.Resampling.BILINEAR)  # or NEAREST, or LANCZOS

    # 轉回 numpy 並正規化
    # Convert back to NumPy and normalize pixel values to [0, 1].
    # 轉回 NumPy 格式並將像素值標準化到 [0, 1]。
    frame = np.asarray(image, dtype=np.float32) / 255.0

    return frame


def stack_frames(stacked_frames, state, is_new_episode):
    # 預處理目前影格
    frame = preprocess_frame(state)

    if is_new_episode or stacked_frames is None:
        # If it's a new episode or no previous frames, initialize with 4 identical frames
        # 若是新的一集或是尚未初始化，則用目前影格複製 4 次形成初始堆疊
        stacked_frames = deque([frame]*4, maxlen=4)
    else:
        # 否則把新影格加入到堆疊中，自動捨棄最舊的
        stacked_frames.append(frame)

    # Stack the 4 frames along the first dimension: shape becomes (4, 84, 84)
    # 沿著第一維（channel）堆疊成 4 通道輸入：形狀變成 (4, 84, 84)
    stacked_state = np.stack(stacked_frames, axis=0)

    return stacked_state, stacked_frames


In [24]:
num_actions = 4  # Breakout 中的動作數量（例如：無動作、左移、右移、發球）  
i_channels = 4  # 堆疊的影格數量（例如：4 個連續影格）
# Number of possible actions in Breakout (e.g., NOOP, LEFT, RIGHT, FIRE)

model = DDQN(i_channels, num_actions).to(device)  
# 建立 DQN 模型並放到指定裝置（CPU 或 GPU）  
# Create a DQN model and move it to the specified device (CPU or GPU)

checkpoint = torch.load('checkpoint_DDQN_V1.pth', map_location=device)
model.load_state_dict(checkpoint['policy_net'])
# 載入訓練好的模型權重（可跨裝置載入）  
# Load trained model weights (supports device mapping for CPU/GPU compatibility)

model.eval()  
# 設定模型為評估模式，關閉 dropout/batchnorm 等訓練特性  
# Set the model to evaluation mode (disables dropout, batchnorm, etc.)


C:\Users\Wei\AppData\Local\Temp\ipykernel_39584\177018745.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint_DDQN_V1.pth', map_location=

DDQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=4, bias=True)
  )
)

In [29]:
# Visualization of trained agent
env = SpaceShipEnv()
env.render()
state_raw = env.reset()
stacked_frames = None
state_b, state = preprocess_state(state_raw, None, True)  # 使用 preprocess_state 初始化
done = False
frames = []

while not done:
    state_tensor = state_b.to(device)
    q_values = model(state_tensor)
    action = torch.argmax(q_values, dim=1).item()

    next_state_raw, reward, done, info = env.step(action)
    next_state_b, next_state = preprocess_state(next_state_raw, state, False)  # 使用 preprocess_state 更新狀態
    state_b, state = next_state_b, next_state

    # 把畫面抓下來（RGB）
    surface = pygame.display.get_surface()
    frame = pygame.surfarray.array3d(surface)  # shape: (W, H, 3)
    frame = np.transpose(frame, (1, 0, 2))     # pygame 是 x,y → imageio 是 y,x
    frames.append(frame)
    pygame.event.pump()

print(f"reward: {reward}, score: {info['score']}")
env.close()

reward: -80.04990217606297, score: 150


In [30]:
print(len(frames))

251


In [31]:
import imageio

video_path = "space_ship_DDQNCNN_V1_2.mp4"

imageio.mimsave(video_path, frames, fps=60, quality=9)
print(f"Saved gameplay video to: {video_path}")

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (250, 300) to (256, 304) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved gameplay video to: space_ship_DDQNCNN_V1_2.mp4
